In [ ]:
# 코랩 한글 설치, 런타임 다시 시작 실행
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

## 0. 기본작업

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyproj -q

     |████████████████████████████████| 6.3 MB 8.1 MB/s 


In [3]:
# 전처리 라이브러리
import pandas as pd
import numpy as np
import os

# 시각화 라이브러리
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# 내부에 결과를 출력하도록 설정
%matplotlib inline
plt.rc('font', family='NanumBarunGothic') 

# 마이너스 기호 출력
plt.rc('axes', unicode_minus=False)

# 분석에 문제가 없는 경고 메시지 숨김
import warnings
warnings.filterwarnings('ignore')

# 지도 좌표계
from pyproj import Proj, transform

# 지도 시각화
import json
import folium
from folium.plugins import MarkerCluster, FeatureGroupSubGroup
from folium.features import DivIcon

# 작업 진행바 불러오기
from tqdm import tqdm

## **1. 행정동 시각화**

In [4]:
# 강서구 지도 정보 불러오기
os.chdir('/content/drive/Shareddrives/강서구/220311 EDA/')
geo_path = 'gangseo.json'
gangseo_geo = json.load(open(geo_path, encoding='utf-8'))

### 1. 행정동별 지하철역 데이터



In [ ]:
# 지하철역 위도, 경도 좌표
subway_dic = {"방화":[37.577446,126.812741], "개화산":[37.572399,126.806171], "송정":[37.561184,126.811973], "마곡":[37.560183,126.825448],
              "발산":[37.558598,126.837668], "우장산":[37.548768,126.836318], "화곡":[37.541513,126.840461], "까치산":[37.531768,126.846683],
              "김포공항":[37.562434,126.801058], "공항시장":[37.563726,126.810678], "신방화":[37.567532,126.816601], "마곡나루":[37.567336,126.829497],
              "양천향교":[37.568381,126.841333], "가양":[37.561391,126.854456], "증미":[37.557402,126.861939], "등촌":[37.550632,126.865689],
              "염창":[37.546936,126.874916], "신목동":[37.544277,126.88308], "개화":[37.578608,126.798153]}

In [ ]:
# 강서구 지도 시각화
m = folium.Map(location=[37.56,126.83], zoom_start=13, tiles='cartodbpositron')

folium.GeoJson(
    gangseo_geo,
    name = '강서구 지도',
    style_function = lambda feature: {
        'fillOpacity': 0.1,
        'weight': 1.5,
        'linecolor': '#E2E2E2'}
).add_to(m)

# 지하철역 마크 생성
for i in subway_dic:
    folium.Marker(location=subway_dic[i],
                  popup=i+'역',
                  tooltip=i+'역',
                  icon = folium.Icon(color = 'black', icon = 'train', prefix = 'fa'),
                  ).add_to(m)

# 범례 생성
folium.LayerControl(collapsed=False).add_to(m)

m

### 2. 행정동별 버스정류소 데이터 (수정 필요)

In [ ]:
bus = pd.read_csv('서울시버스정류소좌표데이터(2021.01.14.).csv', encoding='cp949')
bus2 = bus[(bus['Y좌표']>=37.527338)&(bus['X좌표']<126.866334)]
bus = pd.concat([bus2, bus[(bus['Y좌표']>=37.542623)&(bus['Y좌표']<=37.556195)&(bus['X좌표']<=126.889591)&(bus['X좌표']>=126.866334)]], axis=0)
bus = bus[bus['정류소명']!='화훼단지']
bus = bus[bus['정류소명']!='팔복부동산.목동교회'].reset_index(drop=True)

In [ ]:
# 강서구 지도 시각화
m = folium.Map(location=[37.56,126.83], zoom_start=13, tiles='cartodbpositron')

folium.GeoJson(
    gangseo_geo,
    name = '강서구 지도',
    style_function = lambda feature: {
        'fillOpacity': 0.1,
        'weight': 1.5,
        'linecolor': '#E2E2E2'}
).add_to(m)

# 버스정류소 점 찍기
for i in range(len(bus)):
    folium.Circle(location=[bus['Y좌표'][i],bus['X좌표'][i]],
                  radius=10, # 원 크기
                  color='#eb9e34', # 원 선 색상
                  fill_color='eb9e34', # 원 내부 색상
                  popup=bus['정류소명'][i],
                  tooltip=bus['정류소명'][i]
                  ).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m

### 3. 인구통계 (나이대, 성별)

#### 1. (들어가기 앞서) 중고거래 사용자 분석

- 플랫폼
> 당근마켓을 위주 (직거래 플랫폼이므로), 중고나라는 이용률이 적으므로 사용 X
- 성별
> 당근마켓: 여성 이용자가 약간 더 많음 <br> 번개장터: 남성 이용자가 약간 더 많음 <br> -> 이용률 차이가 극단적이지 않으므로 앱 이용자 성별은 중요한 지표 X <br> 오히려 안전을 중시하는 여성을 타겟으로 하는 게 바람직함
- 연령
> 당근마켓: 40 ~ 50대 이용자가 높게 나타남. <br> 번개장터: 20대, 50대 이용자가 높게 나타남. <br> -> 20대, 40 ~ 50대 이용자를 위주로 살펴볼 것.


In [ ]:
os.chdir('/content/drive/Shareddrives/강서구/220315 DrawMap/data/')
resident = pd.read_csv('행정구역_읍면동_별_5세별_주민등록인구_2011년__20220317090236.csv', encoding='cp949')

# 강서구 행정구역(동읍면)별 20,40,50대 여성 인구수
resident_all = pd.concat([resident[['행정구역(동읍면)별','항목']], resident.iloc[:,55:59],resident.iloc[:,51:53]], axis=1).drop([0,1,2,3,4,5,6], axis=0) # 필요 없는 행, 열 삭제
resident_all = resident_all[resident_all['항목']=='여자인구수 (명)'].reset_index(drop=True) # 필요 없는 행 삭제
resident_all.drop('항목', axis=1, inplace=True) # 필요 없는 열 삭제
resident_all["행정구역(동읍면)별"] = resident_all["행정구역(동읍면)별"].str.replace(pat=r'제', repl=r'', regex=True) # 불필요한 단어 제거
resident_all.set_index('행정구역(동읍면)별',inplace=True) 
resident_all = resident_all.astype('float64') # 변수 타입 변경
resident_all['people'] = resident_all.sum(axis=1) # 20,40,50대 여성 인구수 합계 계산
resident_all.reset_index(inplace=True)
resident_all = resident_all[['행정구역(동읍면)별', 'people']] # 필요없는 열 삭제

# 생활인구 데이터 불러오기
os.chdir('/content/drive/Shareddrives/강서구/220311 EDA/행정구 단위/')
df_people = pd.read_csv('LOCAL_PEOPLE_DONG_202202.csv')

# 한글 columns명이 너무 길어서 영어명(생활인구_스키마_공개 파일 기준)으로 간소화
df_kor = df_people.columns
df_people.reset_index(inplace=True)
df_people = df_people.drop(['여자70세이상생활인구수'], axis=1) # 한칸씩 밀리는 오류 해결
df_people.columns = ['YMD','TT','H_DNG_CD','SPOP',
                       'M00','M10','M15','M20','M25','M30','M35','M40','M45','M50','M55','M60','M65','M70',
                       'F00','F10','F15','F20','F25','F30','F35','F40','F45','F50','F55','F60','F65','F70']

# 강서구의 행정코드는 11500510~11500641
gs_people = df_people[(df_people['H_DNG_CD'] >= 11500510) & (df_people['H_DNG_CD'] <= 11500641)]
gs_people = gs_people.reset_index(drop=True)

# 행정동 이름 및 코드 딕셔너리
gs_lg = ['염창동','등촌1동', '등촌2동', '등촌3동', '화곡1동', '화곡2동', '화곡3동', '화곡4동', '화곡본동', '화곡6동', '화곡8동',
         '가양1동', '가양2동', '가양3동', '발산1동', '우장산동', '공항동','방화1동','방화2동','방화3동']
gs_cd = [11500510,11500520,11500530,11500535,11500540,11500550,11500560,11500570,11500590,11500591,
         11500593,11500603,11500604,11500605,11500611,11500615,11500620,11500630,11500640,11500641]
gs_dict =  {gs_cd[i] : gs_lg[i] for i in range(len(gs_lg))}

# 강서구 20,40,50대 여성 평균 생활인구 시각화
gs_people = gs_people[['YMD','TT','H_DNG_CD','F20','F25','F40','F45','F50','F55']]
gs_people['people'] = gs_people.iloc[:,3:].sum(axis=1)
gs_people = gs_people[['YMD','TT','H_DNG_CD','people']] # 합계
gs_people_mean = gs_people.groupby(['H_DNG_CD']).mean() # 행정동별 평균값
gs_people_mean.drop(['TT','YMD'], axis=1, inplace=True)
gs_people_mean.reset_index(inplace=True)
gs_people_mean['H_DNG_CD'] = gs_people_mean['H_DNG_CD'].apply(lambda x : gs_dict.get(x) if x else x)

In [ ]:
# 강서구 지도 시각화
m = folium.plugins.DualMap(location=[37.56,126.83], zoom_start=13, tiles='cartodbpositron')

# 강서구 행정구역(동읍면)별 총인구수 시각화
f1 = folium.Choropleth(geo_data=gangseo_geo,
                    data=resident_all, 
                    columns = resident_all.columns,
                    key_on='feature.properties.name',
                    fill_color='YlGn',
                    fill_opacity=0.7,
                    line_opacity=0.3,
                    legend_name="20, 40, 50대 여성 주민등록인구수(명)",
                    name = '강서구 지도',
                    highlight=True).add_to(m.m1)

# 강서구 20,40,50대 여성 평균 생활인구 시각화
f2 = folium.Choropleth(geo_data=gangseo_geo,
                    data=gs_people_mean, 
                    columns = gs_people_mean.columns,
                    key_on='feature.properties.name',
                    fill_color='YlGn',
                    fill_opacity=0.7,
                    line_opacity=0.3,
                    legend_name="주20, 40, 50대 여성 생활인구수(명)",
                    name = '강서구 지도',
                    highlight=True).add_to(m.m2)

# 범례 추가
folium.LayerControl(collapsed=False).add_to(m)

# 행정동명 이름 나오게
f1.geojson.add_child(folium.features.GeoJsonTooltip(['name'], labels=False))
f2.geojson.add_child(folium.features.GeoJsonTooltip(['name'], labels=False))

m

## **2. 집계구 시각화**

In [14]:
seoul_geo = json.load(open('/content/drive/Shareddrives/강서구/220315 DrawMap/집계구.json'))

proj_UTMK = Proj(init='epsg:5179')
proj_WGS84 = Proj(init='epsg:4326')

gangsu_list = []
for i in seoul_geo['features']:
  if i['properties']['ADM_CD'].startswith('11160'): # 강서구 데이터인 경우에만
    points = np.array(i['geometry']['coordinates'][0]) # 좌표값 추출

    x,y = points[:,0].tolist(), points[:,1].tolist() # 경도 위도 각각 추출
    new_x, new_y = transform(proj_UTMK, proj_WGS84, x, y) # 좌표계 변환
    new_points = np.column_stack((new_x, new_y)) # 다시 열 병합

    i['geometry']['coordinates'][0] = new_points.tolist() # json 저장을 위한 리스트 변환
    gangsu_list.append(i) # json 데이터에 대입
gangsu_geo = seoul_geo.copy() # 강서구용을 위해 데이터 복사
gangsu_geo['features'] = gangsu_list # 강서구 데이터 생성

In [ ]:
# 지하철역 위도, 경도 좌표
subway_dic = {"방화":[37.577446,126.812741], "개화산":[37.572399,126.806171], "송정":[37.561184,126.811973], "마곡":[37.560183,126.825448],
              "발산":[37.558598,126.837668], "우장산":[37.548768,126.836318], "화곡":[37.541513,126.840461], "까치산":[37.531768,126.846683],
              "김포공항":[37.562434,126.801058], "공항시장":[37.563726,126.810678], "신방화":[37.567532,126.816601], "마곡나루":[37.567336,126.829497],
              "양천향교":[37.568381,126.841333], "가양":[37.561391,126.854456], "증미":[37.557402,126.861939], "등촌":[37.550632,126.865689],
              "염창":[37.546936,126.874916], "개화":[37.578608,126.798153]}

# 버스 정류소
os.chdir('/content/drive/Shareddrives/강서구/220311 EDA/')
bus = pd.read_csv('서울시버스정류소좌표데이터(2021.01.14.).csv', encoding='cp949')
bus = bus[(bus['Y좌표']>37.528055)&(bus['X좌표']<126.877717)]
bus = bus[~((bus['Y좌표']>37.529061)&(bus['Y좌표']<=37.541233)&(bus['X좌표']<=126.833127)&(bus['X좌표']>=126.823362))]
bus = bus[~((bus['Y좌표']>37.528325)&(bus['Y좌표']<=37.546752)&(bus['X좌표']<=126.877377)&(bus['X좌표']>=126.866334))]
bus = bus[(bus['정류소명']!='화훼단지')&(bus['정류소명']!='수명산입구')&(bus['정류소명']!='팔복부동산.목동교회')&
          (bus['정류소명']!='보람쉬움아파트.신영시장')&(bus['정류소명']!='보람쉬움아파트')&(bus['정류소명']!='난지한강공원')&
          (bus['정류소명']!='난지한강공원(야구장)')&(bus['정류소명']!='롯데캐슬아파트.목3동시장')&(bus['정류소명']!='양동중학교')&(bus['정류소명']!='강서교육청')].reset_index(drop=True)

In [ ]:
# 집계구 데이터 불러오기
os.chdir('/content/drive/Shareddrives/강서구/220315 DrawMap/data/GANGSEO_LOCAL_PEOPLE_2202202/')
num = list(map(str, range(1,29))) # 1일 ~ 29일
for i in range(len(num)):
    num[i] = num[i].zfill(2)
for i in num:
    globals()['gs_lp_{}'.format(i)] = pd.read_csv('gs_lp_'+i+'.csv', encoding="CP949")

# 2월 강서구 20,40,50대 여성 데이터 만들기
for i in num:
    globals()['gs_lp_{}'.format(i)] = globals()['gs_lp_{}'.format(i)][['YMD','TT','SMGU_CD','F20','F25','F40','F45','F50','F55']]
gs_lp = pd.concat([globals()['gs_lp_{}'.format(i)] for i in num])

# 행정동 이름 및 코드 딕셔너리
gs_lg = ['염창동','등촌1동', '등촌2동', '등촌3동', '화곡1동', '화곡2동', '화곡3동', '화곡4동', '화곡본동', '화곡6동', '화곡8동',
         '가양1동', '가양2동', '가양3동', '발산1동', '우장산동', '공항동','방화1동','방화2동','방화3동']
gs_cd = [11500510,11500520,11500530,11500535,11500540,11500550,11500560,11500570,11500590,11500591,
         11500593,11500603,11500604,11500605,11500611,11500615,11500620,11500630,11500640,11500641]
gs_dict =  {gs_cd[i] : gs_lg[i] for i in range(len(gs_lg))}

# 강서구 20,40,50대 여성 평균 생활인구
gs_lp['people'] = gs_lp.iloc[:,3:].sum(axis=1)
gs_people = gs_lp[['YMD','TT','SMGU_CD','people']] # 합계
gs_lp_mean = gs_lp.groupby(['SMGU_CD']).mean() # 행정동별 평균값
gs_lp_mean.drop(['TT','YMD'], axis=1, inplace=True) # SPOP 값이 전체합계와 일치하지 않을 수 있음에 주의
gs_lp_mean.reset_index(inplace=True)
gs_lp_mean = gs_lp_mean[['SMGU_CD','people']]
gs_lp_mean['SMGU_CD'] = gs_lp_mean['SMGU_CD'].astype('str')

In [ ]:
len(bus)

611

In [ ]:
# 강서구 지도 시각화
m = folium.Map(location=[37.56,126.83], zoom_start=13, tiles='cartodbpositron')

# 강서구 20,40,50대 여성 평균 생활인구 시각화
f = folium.Choropleth(geo_data=gangsu_geo,
                    data=gs_lp_mean, 
                    columns = gs_lp_mean.columns,
                    key_on='feature.properties.TOT_REG_CD',
                    fill_color='YlGn',
                    fill_opacity=0.7,
                    line_opacity=0.3,
                    legend_name="20, 40, 50대 여성 생활인구수(명)",
                    name = '강서구 집계구 지도').add_to(m)

# 지하철역 마크 생성
for i in subway_dic:
    folium.Marker(location=subway_dic[i],
                  popup=i+'역',
                  tooltip=i+'역',
                  icon = folium.Icon(color = 'black', icon = 'train', prefix = 'fa'),
                  ).add_to(m)

# 버스정류소 점 찍기
for i in range(len(bus)):
    folium.Circle(location=[bus['Y좌표'][i],bus['X좌표'][i]],
                  radius=10, # 원 크기
                  color='#eb9e34', # 원 선 색상
                  fill_color='eb9e34', # 원 내부 색상
                  popup=bus['정류소명'][i],
                  tooltip=bus['정류소명'][i]
                  ).add_to(m)

# 범례 생성
folium.LayerControl(collapsed=False).add_to(m)

# 행정동명 이름 나오게
f.geojson.add_child(folium.features.GeoJsonTooltip(['ADM_NM'], labels=False))

m

Output hidden; open in https://colab.research.google.com to view.

## **3. 기타**

In [ ]:
# (파일이 너무 커서) 강서구 집계구 파일 생성
'''
# 전처리 라이브러리
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# 집계구 데이터 불러오기
os.chdir('/content/drive/Shareddrives/강서구/220315 DrawMap/data/LOCAL_PEOPLE_202202/')

# 20일 ~ 28일
num = list(map(str, range(20,29)))
for i in range(len(num)):
    num[i] = num[i].zfill(2)
for i in tqdm(num):
    globals()['gs_lp_{}'.format(i)] = pd.read_csv('LOCAL_PEOPLE_202202'+i+'.csv', encoding="CP949")

# 강서구의 행정코드는 11500510~11500641
for i in num:
    globals()['gs_lp_{}'.format(i)] = globals()['gs_lp_{}'.format(i)][(globals()['gs_lp_{}'.format(i)]['행정동코드'] >= 11500510) & (globals()['gs_lp_{}'.format(i)]['행정동코드'] <= 11500641)]
    globals()['gs_lp_{}'.format(i)] = globals()['gs_lp_{}'.format(i)].reset_index(drop=True)

# 한글 columns명이 너무 길어서 영어명(생활인구_스키마_공개 파일 기준)으로 간소화
gs_lp_kor = gs_lp_01.columns
for i in num:
    globals()['gs_lp_{}'.format(i)].columns = ['YMD','TT','H_DNG_CD','SMGU_CD','SPOP',
                 'M00','M10','M15','M20','M25','M30','M35','M40','M45','M50','M55','M60','M65','M70',
                 'F00','F10','F15','F20','F25','F30','F35','F40','F45','F50','F55','F60','F65','F70']

# 분포 등을 확인하기 위해 인구수가 3명 이하(*)인 경우 0으로 대체
for i in num:
    for j in range(5, 33):
        globals()['gs_lp_{}'.format(i)].iloc[:,j] = globals()['gs_lp_{}'.format(i)].iloc[:,j].apply(lambda x : 0 if x=='*' else x)
        globals()['gs_lp_{}'.format(i)].iloc[:,j] = globals()['gs_lp_{}'.format(i)].iloc[:,j].astype('float64')

os.chdir('/content/drive/Shareddrives/강서구/220315 DrawMap/data/GANGSEO_LOCAL_PEOPLE_2202202/')

for i in num:
    globals()['gs_lp_{}'.format(i)].to_csv('gs_lp_'+i+'.csv',index = False)
'''

100%|██████████| 9/9 [01:07<00:00,  7.46s/it]


## 거주지

In [5]:
home = pd.read_csv('/content/drive/Shareddrives/강서구/220320 MCLP/data/공동주택_위도경도.csv')

,단 지 명,동명,도로명주소,위도,경도
0,강변삼천리,염창동,양천로67길 71-16,37.556897,126.871275
1,강변한솔솔파크,염창동,양천로63길 38,37.556889,126.866573
2,강변힐스테이트,염창동,양천로75길 19,37.551240,126.874120
3,관음삼성,염창동,양천로 656,37.552711,126.869139
4,극동상록수,염창동,공항대로71길 31,37.549000,126.871933
...,...,...,...,...,...
95,태승훼미리3차,발산1동,강서로45길 138,37.551445,126.831380
96,이너스내안에 301동,염창동,양천로67가길 56,37.553553,126.870450
97,강서쌍용예가,염창동,양천로 613,37.555423,126.866482
98,등촌동 리본타워,등촌3동,공항대로 359,37.557847,126.847820


In [15]:
# 강서구 지도 시각화
m = folium.Map(location=[37.56,126.83], zoom_start=13, tiles='cartodbpositron')

folium.GeoJson(
    gangsu_geo,
    name = '강서구 지도',
    style_function = lambda feature: {
        'fillOpacity': 0.1,
        'weight': 1.5,
        'linecolor': '#E2E2E2'}
).add_to(m)

for i in range(len(home)):
    folium.Circle(location=[home['위도'][i],home['경도'][i]],
                  radius=10, # 원 크기
                  color='#eb9e34', # 원 선 색상
                  fill_color='eb9e34', # 원 내부 색상
                  popup=home['단 지 명'][i],
                  tooltip=home['단 지 명'][i]
                  ).add_to(m)

# 범례 생성
folium.LayerControl(collapsed=False).add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.